# Проект Яндекс.Музыка

## Описание

На данных Яндекс.Музыки провести исследование поведения пользователей двух столиц Москвы и Санкт-Петербурга.

## Цель исследования

Проверка гипотез:

    1. Активность пользователей зависит от дня недели.
    
    2. Преобладание различных жанров в Москве в Санкт-Петербурге в понедельник утром и вечером в пятницу. 
    
    3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

## Выводы

1. Первая гипотеза полностью подтвердилась: день недели по-разному влияет на активность пользователей в Москве и Петербурге. 


2. Вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных:

    Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

3. Третья гипотеза не подтвердилась:

    Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.
Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

** Из данных одного сервиса не всегда можно сделать вывод о всех жителях города.

## Обзор данных

In [1]:
import pandas as pd # импорт библиотеки pandas

In [2]:
df = pd.read_csv('yandex_music_project.csv') 

In [3]:
#посмотрим данные
df.head(5) 

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


In [4]:
# посмотрим информацию о данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны три нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Отсутствие "змеиного регистра".

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

## Предобработка данных

### Обработка полей

In [5]:
# посмотрим названия столбцов
df.columns 

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [6]:
# переименование столбцов
df = df.rename(columns = {'  userID': 'user_id', 'Track': 'track', '  City  ': 'city', 'Day': 'day'})
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Обработка пропущенных

In [7]:
# подсчёт пропусков
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Пропуски в `track` и `artist` пропуски не важны для нашего исследования. Достаточно заменить их явными обозначениями.

Пропуски в `genre` могут помешать сравнению музыкальных вкусов пользователей. Так так установить причину пропусков нет возможности заменим их на явные обозначения.

In [8]:
# посчитаем процент пропущенных значений жанра

print('Процент пропущенных знаечний:', df.isna().sum()['genre'] /  len(df) * 100, '%')

Процент пропущенных знаечний: 1.8408395949538254 %


In [9]:
# выберем поля для заполнения
columns_to_replace = ['track', 'artist', 'genre']

# заменим пропущенные значение на "unknown"
for i in columns_to_replace:
    df[i] = df[i].fillna('unknown')

In [10]:
# проверим пропуски после обработки
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Дубликаты

#### Явные дубликаты

In [11]:
# посчитаем явные дубликаты
df.duplicated().sum() 

3826

In [12]:
# удалим дубликаты (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True) 

In [13]:
# проверим отсутствие дубликатов после обработки
df.duplicated().sum()

0

#### Неявные дубликаты

In [14]:
# посмотрим на значения в поле 'genre'
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

В данных есть неявные дубликаты или записи с ошибками. Обработаем их.

In [15]:
# напишем функцию для замены некорректных значений на корректные

def replace_wrong_genres(wrong_genres, correct_genre):
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genre, correct_genre)

In [16]:
# заменим некорректные значения

wrong_genres = ['hip', 'hop', 'hip-hop']             # некорректные названия
correct_genre = 'hiphop'                             # корректное название
replace_wrong_genres(wrong_genres, correct_genre)    # замена

In [17]:
# заменим одно некорректное название

df ['genre'] = df ['genre'].replace('электроника', 'electronic')

In [18]:
# проверим на неявные дубликаты после обработки

df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

## Проверка гипотез

### 1 гипотеза. Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.

In [19]:
# посчитаем прослушивания в каждом городе

df.groupby('city')['user_id'].count().reset_index()

,city,user_id
0,Moscow,42741
1,Saint-Petersburg,18512


В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

In [20]:
# посчитаем прослушивания в каждый из трёх дней

df.groupby('day').count()['user_id'].reset_index()

,day,user_id
0,Friday,21840
1,Monday,21354
2,Wednesday,18059


В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

In [21]:
# создадим функцию number_tracks, которая посчитает прослушивания для заданного дня и города

def  number_tracks(day, city):
    track_list = df [(df['day'] == day) & (df['city'] == city)]
    track_list_count = track_list['user_id'].count()
    return track_list_count

In [22]:
# количество прослушиваний в Москве по понедельникам
print('Количество прослушиваний в Москве в понедельник:', number_tracks(day = 'Monday', city = 'Moscow'))  

# количество прослушиваний в Санкт-Петербурге по понедельникам
print('Количество прослушиваний в Санкт-Петербурге в понедельник:', number_tracks(day = 'Monday', city = 'Saint-Petersburg')) 
print('')

# количество прослушиваний в Москве по средам
print('Количество прослушиваний в Москве в среду:', number_tracks(day = 'Wednesday', city = 'Moscow')) 

# количество прослушиваний в Санкт-Петербурге по средам
print('Количество прослушиваний в Санкт-Петербурге в среду:', number_tracks(day = 'Wednesday', city = 'Saint-Petersburg')) 
print('')

# количество прослушиваний в Москве по пятницам
print('Количество прослушиваний в Москве в пятницу:', number_tracks(day = 'Friday', city = 'Moscow'))

# количество прослушиваний в Санкт-Петербурге по пятницам
print('Количество прослушиваний в Санкт-Петербурге в пятницу:', number_tracks(day = 'Friday', city = 'Saint-Petersburg'))

Количество прослушиваний в Москве в понедельник: 15740
Количество прослушиваний в Санкт-Петербурге в понедельник: 5614

Количество прослушиваний в Москве в среду: 11056
Количество прослушиваний в Санкт-Петербурге в среду: 7003

Количество прослушиваний в Москве в пятницу: 15945
Количество прослушиваний в Санкт-Петербурге в пятницу: 5895


In [23]:
# построим таблицу для наглядности 

columns = ['city', 'monday', 'wednesday', 'friday']
data = [['Moscow',\
         number_tracks(day = 'Monday', city = 'Moscow'),\
         number_tracks(day = 'Wednesday', city = 'Moscow'),\
         number_tracks(day = 'Friday', city = 'Moscow')],\
        ['Saint-Petersburg',\
         number_tracks(day = 'Monday', city = 'Saint-Petersburg'),\
         number_tracks(day = 'Wednesday', city = 'Saint-Petersburg'),\
         number_tracks(day = 'Friday', city = 'Saint-Petersburg')]]
pd.DataFrame(data = data, columns = columns)

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


### Выводы


Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### 2 гипотеза. Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Разделим данные на Москву и Санкт-Петербург

In [24]:
# данные по Москве

moscow_general = df[df['city'] == 'Moscow']
moscow_general

,user_id,track,artist,genre,city,time,day
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,unknown,ruspop,Moscow,09:17:40,Friday
...,...,...,...,...,...,...,...
61247,83A474E7,I Worship Only What You Bleed,The Black Dahlia Murder,extrememetal,Moscow,21:07:12,Monday
61248,729CBB09,My Name,McLean,rnb,Moscow,13:32:28,Wednesday
61250,C5E3A0D5,Jalopiina,unknown,industrial,Moscow,20:09:26,Friday
61251,321D0506,Freight Train,Chas McDevitt,rock,Moscow,21:43:59,Friday


In [25]:
# данные по Санкт-Петербургу

spb_general = df[df['city'] == 'Saint-Petersburg']

Создайте функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [26]:
# Напишем функцию genre_weekday с параметрами table, day, time1, time2,
# которая возвращает информацию о самых популярных жанрах в указанный день в
# заданное время:

def genre_weekday(table, day, time1, time2):
    # в переменную genre_df сохраняются те строки переданного датафрейма table, для
    # которых одновременно:
        # - значение в столбце day равно значению аргумента day
        # - значение в столбце time больше значения аргумента time1
        # - значение в столбце time меньше значения аргумента time2
    genre_df = table[(table['day'] == day) & (table['time'] > time1) & (table['time'] < time2)]
    
    # сгруппируем genre_df по столбцу genre, возьмем один из его
    # столбцов и посчитать методом count() количество записей для каждого из
    # присутствующих жанров, получившийся Series запишем в переменную
    # genre_df_count
    genre_df_count = genre_df.groupby('genre')['genre'].count()
    
    # отсортируем genre_df_count по убыванию встречаемости и сохраним
    # в переменную genre_df_sorted
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    
    # вернуем Series из 10 первых значений genre_df_sorted, это будут топ-10
    # популярных жанров (в указанный день, в заданное время)
    return genre_df_sorted.head(10)

Посмотрим на данные для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [27]:
# Москва, понедельник с 7 до 11 утра

print('Москва с 7 до 11 утра в понедельник:')
print('')
print(genre_weekday(moscow_general, 'Monday', '07:00', '11:00'))

Москва с 7 до 11 утра в понедельник:

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64


In [28]:
# Санкт-Петербург, понедельник с 7 до 11 утра

print('Санкт-Петербург с 7 до 11 утра в понедельник:')
print('')
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

Санкт-Петербург с 7 до 11 утра в понедельник:



genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [29]:
# Москва, пятница с 17 до 23 вечера

print('Москва с 17 до 23 вечера в пятницу:')
print('')
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

Москва с 17 до 23 вечера в пятницу:



genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [30]:
# Санкт-Петербург, пятница с 17 до 23 вечера

print('Санкт-Петербург с 17 до 23 вечера в пятницу:')
print('')
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

Санкт-Петербург с 17 до 23 вечера в пятницу:



genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

### Выводы


Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### 3 гипотеза. Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

In [31]:
# сругппируем Москву по жанру, посчитаем и отсортируем
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)

# посмотрим топ-10 жанров Москвы
print('Топ-10 жанров Москвы')
print('')
print(moscow_genres.head(10))

Топ-10 жанров Москвы

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64


In [32]:
# сругппируем Санкт-Петербург по жанру, посчитаем и отсортируем
spb_general = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)

# посмотрим топ-10 жанров Москвы
print('Топ-10 жанров Санкт-Петербурга')
print('')
print(spb_general.head(10))

Топ-10 жанров Санкт-Петербурга

genre
pop            2431
dance          1932
rock           1879
electronic     1737
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64


### Выводы

Гипотеза не подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 
